## Opening the files.

In [1]:
import mne
import glob

DATA_DIR = "/home/sysadmin/Martin/opm-thesis/data/gestures/"
raw_files = glob.glob(DATA_DIR + "/*-raw.fif.gz")
raw = mne.io.read_raw_fif(raw_files[0], preload=True)

# Epochs
epoch_files = glob.glob(DATA_DIR + "/*-epo.fif.gz")


Opening raw data file /home/sysadmin/Martin/opm-thesis/data/gestures/meg_rps-run_1-raw.fif.gz...
Isotrak not found
    Read a total of 8 projection items:
        HFC: l=1 m=-1 (1 x 147) active
        HFC: l=1 m=0 (1 x 147) active
        HFC: l=1 m=1 (1 x 147) active
        HFC: l=2 m=-2 (1 x 147) active
        HFC: l=2 m=-1 (1 x 147) active
        HFC: l=2 m=0 (1 x 147) active
        HFC: l=2 m=1 (1 x 147) active
        HFC: l=2 m=2 (1 x 147) active
    Range : 0 ... 833759 =      0.000 ...   694.799 secs
Ready.
Reading 0 ... 833759  =      0.000 ...   694.799 secs...


## CSP

In [2]:
data_epochs = "/home/sysadmin/Martin/opm-thesis/data/gestures_epochs/freq_bands/all_data_all_epochs_decimated.pkl"


In [7]:
"""This script is used to classify the data using CSP and Logistic Regression.

This classifier takes the epochs from the different frequency bands and
acquisition times and trains a binary CSP classifier for each possible pair of
ids. Then, it predicts the label of each epoch using each classifier and gives the
results of the pairs.
"""
import pickle
import mne
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP
from matplotlib import pyplot as plt


# Define possible pairs for classification
ids = [2, 4]
DATA_DIR = "./data/"
EPOCHS_PATH = DATA_DIR + "gestures_epochs/freq_bands/all_data_all_epochs_decimated.pkl"
with open(EPOCHS_PATH, "rb") as f:
    epochs = pickle.load(f)

total_scores = {}
USE_Z = False

# Indices whwenever the id is 1 or 2
indices = np.where(np.isin(epochs.events[:, 2], ids))[0]
all_epochs = epochs[indices]

picks = mne.pick_types(all_epochs.info, meg=True, exclude="bads")
id_epochs = all_epochs.copy().pick(picks)

if USE_Z:
  selected_chs = [ch for ch in epochs.ch_names if "[Z]" in ch]
  id_epochs.pick(selected_chs)

# Normalize
data = np.real(id_epochs.get_data())
data -= np.mean(data, axis=(0, 2), keepdims=True)
data /= np.std(data, axis=(0, 2), keepdims=True)

labels = id_epochs.events[:, 2]

# Building classifier
clf = LinearDiscriminantAnalysis()

# Split data into training and test sets once
train_idx, test_idx = train_test_split(
    np.arange(data.shape[0]),  # indices to split
    test_size=0.2,  # 20% test size
    random_state=50,  # seed for reproducibility
    stratify=labels,  # preserve label balance
)

# Use the indices to select the train/test data
X_train, y_train = data[train_idx], labels[train_idx]
X_test, y_test = data[test_idx], labels[test_idx]

# Fit CSP on training data
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False).fit(X_train, y_train)

# Transform training and test data
X_train_csp = csp.transform(X_train)
X_test_csp = csp.transform(X_test)

# Train and evaluate classifier
clf.fit(X_train_csp, y_train)
score = clf.score(X_test_csp, y_test)*100

print(f"Score: {score:.2f}")

csp.plot_patterns(id_epochs.info, ch_type="mag", units="Patterns (AU)", size=1.5)
plt.show()


Computing rank from data with rank=None
    Using tolerance 50 (2.2e-16 eps * 147 dim * 1.5e+15  max singular value)
    Estimated rank (mag): 147
    MAG: rank 147 computed from 147 data channels with 0 projectors
Reducing data rank from 147 -> 147
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 49 (2.2e-16 eps * 147 dim * 1.5e+15  max singular value)
    Estimated rank (mag): 147
    MAG: rank 147 computed from 147 data channels with 0 projectors
Reducing data rank from 147 -> 147
Estimating covariance using EMPIRICAL
Done.
Score: 98.96


ValueError: The following electrodes have overlapping positions, which causes problems during visualization:
FS[Z], FS[Y], FS[X], FT[Z], FT[Y], FT[X], FU[Z], FU[Y], FU[X], FV[Z], FV[Y], FV[X], FW[Z], FW[Y], FW[X], FX[Z], FX[Y], FX[X], FY[Z], FY[Y], FY[X], FZ[Z], FZ[Y], G0[Z], G0[Y], G2[Z], G2[Y], G2[X], HE[Z], HE[Y], HE[X], HF[Z], HF[X], HG[Z], HG[Y], HG[X], HH[Z], HH[Y], HH[X], HI[Z], HI[Y], HI[X], HK[Z], HK[Y], HK[X], HL[Z], HL[Y], HL[X], HN[Z], HN[Y], HN[X], HO[Z], HO[Y], HO[X], HQ[Z], HQ[Y], HQ[X], I0[Z], I0[Y], I0[X], I1[Z], I1[Y], I1[X], I2[Z], I2[Y], I2[X], I3[Z], I3[Y], I3[X], I4[Z], I4[Y], I4[X], I5[Z], I5[Y], I5[X], I6[Z], I6[Y], I6[X], I7[Z], I7[Y], I7[X], K7[Z], K7[Y], K7[X], K8[Z], K8[Y], K8[X], K9[Z], K9[Y], KB[Z], KB[Y], KB[X], KE[Z], KE[X], KF[Z], KF[X], KG[Z], KG[Y], KG[X], KH[Z], KH[Y], KH[X], KI[Z], KI[Y], KI[X], LC[Z], LC[Y], LC[X], LF[Z], LF[Y], LG[Z], LG[Y], LG[X], LH[Z], LH[Y], LH[X], LI[Z], LI[Y], LI[X], LJ[Z], LJ[Y], LJ[X], LK[Z], LK[Y], LK[X], LL[Z], LL[Y], LL

In [12]:
"""This script is used to classify the data using CSP and Logistic Regression.

This classifier takes the epochs from the different frequency bands and
acquisition times and trains a binary CSP classifier for each possible pair of
ids. Then, it predicts the label of each epoch using each classifier and gives the
results of the pairs.
"""
import pickle
import mne
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP
from matplotlib import pyplot as plt


# Define possible pairs for classification
ids = [2, 4]
DATA_DIR = "./data/"
EPOCHS_PATH = DATA_DIR + "gestures_epochs/freq_bands/all_data_all_epochs_decimated.pkl"
with open(EPOCHS_PATH, "rb") as f:
    epochs = pickle.load(f)

# Indices whwenever the id is 1 or 2
indices = np.where(np.isin(epochs.events[:, 2], ids))[0]
all_epochs = epochs[indices]

# Mean over channels
unique_sensor_names = set(ch.rsplit("[", 1)[0] for ch in epochs.ch_names if "[" in ch)
averaged_data = []
averaged_ch_names = []
averaged_ch_info = []

for base_name in unique_sensor_names:
  # Find all components of this sensor
  component_indices = [
      i for i, ch_name in enumerate(epochs.ch_names) if ch_name.startswith(base_name)
  ]
  component_data = epochs.get_data()[:, component_indices, :]

  # Average the components
  avg_data = np.mean(component_data, axis=1)
  averaged_data.append(avg_data)
  ch_info = epochs.info["chs"][
      component_indices[0]
  ].copy()  # Copy info from one of the components
  ch_info["ch_name"] = base_name  # Update the channel name to the base name
  averaged_ch_names.append(base_name)
  averaged_ch_info.append(ch_info)

# Rebuild the Epochs data with averaged components
averaged_data = np.stack(
    averaged_data, axis=1
)  # Stack to match the shape (n_epochs, n_channels, n_times)
averaged_info = mne.create_info(averaged_ch_names, epochs.info["sfreq"], ch_types="mag")
averaged_epochs = mne.EpochsArray(
    averaged_data,
    averaged_info,
    events=epochs.events,
    event_id=epochs.event_id,
    tmin=epochs.tmin,
)

# Normalize
data = np.real(averaged_epochs.get_data())
data -= np.mean(data, axis=(0, 2), keepdims=True)
data /= np.std(data, axis=(0, 2), keepdims=True)

labels = averaged_epochs.events[:, 2]

# Building classifier
clf = LinearDiscriminantAnalysis()

# Split data into training and test sets once
train_idx, test_idx = train_test_split(
    np.arange(data.shape[0]),  # indices to split
    test_size=0.2,  # 20% test size
    random_state=50,  # seed for reproducibility
    stratify=labels,  # preserve label balance
)

# Use the indices to select the train/test data
X_train, y_train = data[train_idx], labels[train_idx]
X_test, y_test = data[test_idx], labels[test_idx]

# Fit CSP on training data
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False).fit(X_train, y_train)

# Transform training and test data
X_train_csp = csp.transform(X_train)
X_test_csp = csp.transform(X_test)

# Train and evaluate classifier
clf.fit(X_train_csp, y_train)
score = clf.score(X_test_csp, y_test) * 100

print(f"Score: {score:.2f}")

csp.plot_patterns(averaged_epochs.info, ch_type="mag", units="Patterns (AU)", size=1.5)
plt.show()


Not setting metadata
718 matching events found
No baseline correction applied
0 projection items activated
Computing rank from data with rank=None
    Using tolerance 12 (2.2e-16 eps * 55 dim * 9.7e+14  max singular value)
    Estimated rank (mag): 55
    MAG: rank 55 computed from 55 data channels with 0 projectors
Reducing data rank from 55 -> 55
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 12 (2.2e-16 eps * 55 dim * 9.9e+14  max singular value)
    Estimated rank (mag): 55
    MAG: rank 55 computed from 55 data channels with 0 projectors
Reducing data rank from 55 -> 55
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 12 (2.2e-16 eps * 55 dim * 9.6e+14  max singular value)
    Estimated rank (mag): 55
    MAG: rank 55 computed from 55 data channels with 0 projectors
Reducing data rank from 55 -> 55
Estimating covariance using EMPIRICAL
Done.
